https://docs.langchain.com/oss/python/langchain/knowledge-base


Percebi que com textos em portguês não foi possível localizar o conteúdo por nenhum dos métodos do langchain-core. preciso entender se o que aconteceu. Criei outro arquivo com pdf em inglês e perguntas também e inglês e funcionou bem.

In [1]:
pip install langchain-community pypdf langchain-openai ipywidgets langchain-core -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

apikeyGpt = os.environ.get('LANGSMITH_API_KEY')
if not apikeyGpt:
    raise RuntimeError("LANGSMITH_API_KEY not set. Set it in PowerShell or Windows Environment Variables.")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = apikeyGpt


In [3]:
## Loading documents
from langchain_community.document_loaders import PyPDFLoader

file_path = "./arquivos/FX2000.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

print('Paginas:' +  str(len(docs)))
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)


Paginas:2
FIRE AND VOICE ALARM SYSTEMS   SGTEXT   June 2019
The FX2000 is an entry level intelligent addressable 
control panel which can be configured for either 1 or 2 
loop operation, and is certified to EN5

{'producer': 'Adobe PDF Library 11.0', 'creator': 'Adobe InDesign CC 2014 (Windows)', 'creationdate': '2015-03-20T12:33:15+00:00', 'moddate': '2019-06-20T09:43:18+01:00', 'trapped': '/False', 'source': './arquivos/FX2000.pdf', 'total_pages': 2, 'page': 0, 'page_label': '3'}


In [4]:
##Splitting documents into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

print(f"{all_splits[3].page_content}")

6
template allowing the mounting screw holes to be pre-drilled and the 
top two mounting screws 
fitted prior to final positioning of the back 
box.
Ample room is provided within the back box for the termination of 
external cables and bonding of cable screen wires.
System configuration
The FX2000 can be configured to operate as a 1 or 2 loop panel. 
Using Eaton system protocol the panel “auto learns” the equipment 
mounted on the loop(s) and “soft addresses” each item, ensuring a 
simple setup procedure and reducing the time required for system 
setup.
All loop mounted equipment has integral short circuit isolators, 
ensuring loop integrity is maintained should a short circuit fault occur 
on the loop.
A comprehensive range of interface units are available and compatible 
with the FX2000.
Standard panel connections
Catalogue numbers
Code Description
MFALOG Fire alarm system log book
Dimensions
H (mm) W (mm) D (mm)
400 320 170
W
H
D
Code FX2000
Description 2 loop control panel


In [5]:
apikeyGpt = os.environ.get('GPT_APIKEY')
if not apikeyGpt:
    raise RuntimeError("GPT_APIKEY not set. Set it in PowerShell or Windows Environment Variables.")

os.environ["OPENAI_API_KEY"] = apikeyGpt

from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_2)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[-0.021003682166337967, 0.04130164161324501, -0.01944100856781006, 0.010543848387897015, 0.009812920354306698, 0.025170812383294106, -0.028850657865405083, 0.036529604345560074, -0.039621345698833466, 0.01772710680961609]


In [6]:
## Creating a vector store InMemory
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

# indexing the document splits
ids = vector_store.add_documents(documents=all_splits)

In [9]:
# performing a similarity search
results = vector_store.similarity_search(
 ##   "Quero usar o nobreak FXP2000 em equipamentos hospitalares. Posso fazer isso?"
 "What is the operating tempetarute of the FX2000?"
)

print(results[0])

page_content='Mains input voltage 230V ac +10% / -15%
Operating voltage 24V dc
Batteries 2 x 12V 7Ah
Standby duration 24 hours (dependant on loop loading)
Environmental
Operating temperature -5°C to +40°C
Humidity (non condensing)
0 to 93% (+2/-3%) RH
Physical
Construction Back box - mild steel, door - PC/ABS
Colour Graphite
Dimensions (H x W x D)
400mm x 320mm x 170mm
Weight 15kg
Ingress protection (with batteries) IP30
Cable entry Top: 20 entry cable knockouts (20mm) 
Rear: 7 entry cable knockouts (20mm)
T echnical specification
FX2000 Intelligent addressable 2 loop control panel910188FULL-0199
JSB product datasheet 
Addressable fire products' metadata={'producer': 'Adobe PDF Library 11.0', 'creator': 'Adobe InDesign CC 2014 (Windows)', 'creationdate': '2015-03-20T12:33:15+00:00', 'moddate': '2019-06-20T09:43:18+01:00', 'trapped': '/False', 'source': './arquivos/FX2000.pdf', 'total_pages': 2, 'page': 1, 'page_label': '4', 'start_index': 2387}


In [11]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score( "What is the operating tempetarute of the FX2000?" )
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.47261271103574937

page_content='Mains input voltage 230V ac +10% / -15%
Operating voltage 24V dc
Batteries 2 x 12V 7Ah
Standby duration 24 hours (dependant on loop loading)
Environmental
Operating temperature -5°C to +40°C
Humidity (non condensing)
0 to 93% (+2/-3%) RH
Physical
Construction Back box - mild steel, door - PC/ABS
Colour Graphite
Dimensions (H x W x D)
400mm x 320mm x 170mm
Weight 15kg
Ingress protection (with batteries) IP30
Cable entry Top: 20 entry cable knockouts (20mm) 
Rear: 7 entry cable knockouts (20mm)
T echnical specification
FX2000 Intelligent addressable 2 loop control panel910188FULL-0199
JSB product datasheet 
Addressable fire products' metadata={'producer': 'Adobe PDF Library 11.0', 'creator': 'Adobe InDesign CC 2014 (Windows)', 'creationdate': '2015-03-20T12:33:15+00:00', 'moddate': '2019-06-20T09:43:18+01:00', 'trapped': '/False', 'source': './arquivos/FX2000.pdf', 'total_pages': 2, 'page': 1, 'page_label': '4', 'start_index': 2387}


In [12]:
embedding = embeddings.embed_query("What is the operating tempetarute of the FX2000?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Mains input voltage 230V ac +10% / -15%
Operating voltage 24V dc
Batteries 2 x 12V 7Ah
Standby duration 24 hours (dependant on loop loading)
Environmental
Operating temperature -5°C to +40°C
Humidity (non condensing)
0 to 93% (+2/-3%) RH
Physical
Construction Back box - mild steel, door - PC/ABS
Colour Graphite
Dimensions (H x W x D)
400mm x 320mm x 170mm
Weight 15kg
Ingress protection (with batteries) IP30
Cable entry Top: 20 entry cable knockouts (20mm) 
Rear: 7 entry cable knockouts (20mm)
T echnical specification
FX2000 Intelligent addressable 2 loop control panel910188FULL-0199
JSB product datasheet 
Addressable fire products' metadata={'producer': 'Adobe PDF Library 11.0', 'creator': 'Adobe InDesign CC 2014 (Windows)', 'creationdate': '2015-03-20T12:33:15+00:00', 'moddate': '2019-06-20T09:43:18+01:00', 'trapped': '/False', 'source': './arquivos/FX2000.pdf', 'total_pages': 2, 'page': 1, 'page_label': '4', 'start_index': 2387}
